In [4]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
import requests
from requests_oauthlib import OAuth1
from os.path import expanduser
import time
home = expanduser("~")

In [59]:
# # Importing our Config

# import cnfg
# config = cnfg.load(home + "/.twitter_config")

# oauth = OAuth1(config["consumer_key"],
#                config["consumer_secret"],
#                config["access_token"],
#                config["access_token_secret"])

In [60]:
# def get_all_tweets(query, batch_count, iter_count):
#     parameters = {"q": query, "count": batch_count}
#     base_url = "https://api.twitter.com/1.1/search/tweets.json"
#     response = requests.get(base_url,
#                         params = parameters,
#                         auth=oauth)
#     tweets = response.json()['statuses']
#     i = 0
#     while i < iter_count - 1:
#         try:
#             response = requests.get(base_url + response.json()['search_metadata']['next_results'],
#                                 auth=oauth)
#             tweets.extend(response.json())
#             print (i)
#             print (len(tweets))
#             i += 1
#         except:
#             return tweets
#             time.sleep(900)
#     return tweets

In [61]:
# tweets = get_all_tweets('budlight', 100, 15)

First we will use the tweepy package to authenticate and connect to the twitter API

In [11]:
# import tweepy

# auth = tweepy.OAuthHandler(config["consumer_key"],
#                            config["consumer_secret"])
# auth.set_access_token(config["access_token"],
#                       config["access_token_secret"])

# api=tweepy.API(auth)

In [12]:
# query = 'budlight'
# max_tweets=2000
# attempts = 50
# attempt = 0

In [ ]:
# retry = True
# while retry:
#     try:
#         retry = False
#         tweets = tweepy.Cursor(api.search,q=query,lang="en").items(max_tweets)
#         for tweet in tweets:
#             results.append(tweet)
#     except tweepy.TweepError:
#         retry = True
#         time.sleep(60)

For now we have about 5000 tweets

In [25]:
# pd.to_pickle(results,'results_budlight.pkl')

In [8]:
results = pd.read_pickle('results_budlight.pkl')

In [10]:
len(results)

/Users/nathanbackblaze/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/Users/nathanbackblaze/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/Users/nathanbackblaze/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/Users/nathanbackblaze/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/Users/nathanbackblaze

5155

In [11]:
results[115].text

u'RT @budlight: The new #BudLightParty look is now all across America, even under your seat. https://t.co/hqBVP48DWC'

The twitter api returns an array of hastags, let's organize this into a set of all the hashtags

In [12]:
def get_hashtags(results):
    hashtags_list = []
    for result in results:
        hashtags = result.entities['hashtags']
        for hashtag in hashtags:
            hashtags_list.append(hashtag['text'].lower())
    return hashtags_list

In [13]:
hashtags_set = get_hashtags(results)

In [14]:
hashtags_set = list(set(hashtags_set))

In [15]:
df = pd.DataFrame(hashtags_set, columns=['hashtags'])

In [17]:
for result in results[:5]:
    print result.text

I'm tipsy in the after math of my two hour long stream, from ONE 8oz can of budlight strawberita. The hell?
Captain Budlight in this beezy https://t.co/1tKcuZ8Cvg
If he is @JohnCollins it will be the last cool one in a long time. So, I'd hope he'd choose something better than a Budlight. 
@Itsjoeco
RT @gabivers: Budlight apple, I could marry you
It's Copa Day! Mexico vs. Venezuela starting at 5pm! We have $3 beers of Montejo, Budlight and Coors! ⚽󾦇... https://t.co/FMQlqU4Ag0


In [20]:
def get_hash_corpus(row):
    row = row.lower()
    corpus = u''
    for result in results:
        hashtag_list = []
        for sub_tag in result.entities['hashtags']:
            hashtag_list.append(sub_tag['text'].lower())
        if row in hashtag_list:
            corpus += result.text
    return corpus

In [21]:
df['corpus'] = [get_hash_corpus(row) for row in df['hashtags']]

In [27]:
df.tail()

,hashtags,corpus
740,watchparty,Join us tonight for a #Cubs #watchparty with #...
741,petesonpike,Starting vacation #1 off right. #petesonpike #...
742,ProGayAgenda,Look at all the bigot #MORONS turning against ...
743,coldbrew,It's a cold bud light kind of night. #budlight...
744,southcarolina,That's how we do a #budlight #yall #southcarol...


In [1]:
pd.to_pickle(df,'budlight_hashed.pkl')

NameError: name 'pd' is not defined

In [5]:
df = pd.read_pickle('budlight_hashed.pkl')

In [6]:
df.tail()

/Users/nathanbackblaze/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/Users/nathanbackblaze/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/Users/nathanbackblaze/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/Users/nathanbackblaze/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/Users/nathanbackblaze

,hashtags,corpus
740,watchparty,Join us tonight for a #Cubs #watchparty with #...
741,petesonpike,Starting vacation #1 off right. #petesonpike #...
742,ProGayAgenda,Look at all the bigot #MORONS turning against ...
743,coldbrew,It's a cold bud light kind of night. #budlight...
744,southcarolina,That's how we do a #budlight #yall #southcarol...


We now have a data frame where the 'hashtags' column has all the hastags mentioned in the budlight twitter query, and 'corpus' has all the text from all the tweets containing the respective hashtag.

This will essentially tell us which text is used in relation to each hashtag, we can now use tfidf and Latent Semantic Analysis to find common topics

For this portion I choose to use the Textacy library (which is built on spaCy) for a few reasons:
- Textacy/spaCy run in Cython, so it is high-performing.  This import because the application will eventually be user-facing
- Textacy/spaCy can handle odd terms such as emojis
- Textacy has an implementation for TFIDF (among other features), which might be import for filtering out unimport words in the terms matrix
- Textacy has Latent Semantic Analysis built it

In [7]:
import textacy

/Users/nathanbackblaze/anaconda/lib/python2.7/site-packages/numpy/lib/utils.py:99: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  warnings.warn(depdoc, DeprecationWarning)


In [8]:
docs = df['corpus']

In [9]:
corpus = textacy.TextCorpus.from_texts('en',docs)

In [10]:
corpus[:5]

[TextDoc(201 tokens; "RT @FrankGirardot: If @Budweiser is now called ..."),
 TextDoc(149 tokens; "RT @KevinGarifo: It's #Friday, so must be time ..."),
 TextDoc(49 tokens; "I need to get out of my own head before it suff..."),
 TextDoc(61 tokens; "@budlight How can I win the @TheChainsmokers ti..."),
 TextDoc(44 tokens; "#MillerLite / #BudLight #hats for sale on #Ebay...")]

In [11]:
print corpus

TextCorpus(745 docs; 159436 tokens)


In [12]:
doc_term_matrix, id2term = corpus.as_doc_term_matrix((doc.as_terms_list(words=True, ngrams=False, named_entities=True)for doc in corpus),weighting='tfidf', normalize=True, smooth_idf=True, min_df=2, max_df=0.95)

In [13]:
print(repr(doc_term_matrix))

<745x2927 sparse matrix of type '<type 'numpy.float64'>'
	with 19252 stored elements in Compressed Sparse Row format>


Now that the tfidf matrix has been, built, we can do topic modeling to see if we can gain any insights on our groups of tweeters.  At this point we can choose between Latent Semantic Analysis (LSA), Latent Dirichlet Allocation (LDA), and Non-negative Matrix Factorization (NMF).  Here is the breakdown for why I will use LSA:

- NMF is generally suited for non text data such as pixels, and NMF can return incoherent topics
- LDA provides a topical distribution for each topic and term and often is more accurate than LSA
- LSA performs Singular Value Decomposition (SVD), which reduces the dimensionality of our already extremely large matrix.  LSA will also execute more quickly which is again important for the end user

In [14]:
model = textacy.tm.TopicModel('lsa', n_topics=10)

In [15]:
model.fit(doc_term_matrix)

In [16]:
doc_topic_matrix = model.transform(doc_term_matrix)

In [17]:
print(doc_topic_matrix.shape)

(745, 10)


In [18]:
for topic_idx, top_terms in model.top_topic_terms(id2term, top_n=10):
    print('topic', topic_idx, ':', '   '.join(top_terms))

('topic', 0, ':', u'budlight   tonight   $   beer   bud   @budlight   budweiser   &amp   3   5')
('topic', 1, ':', u'nba   detroit   pistons   sport   banner   home   bar   hang   bud   2-sid')
('topic', 2, ':', u'$   3   &amp   2   @budlight   5   nba   bar   12:30pm   3.50')
('topic', 3, ':', u'@budlight   today   cmafest   cover   princetribute   realmenofgenius   singer   purplerain   rocknroll   accoustic')
('topic', 4, ':', u'tonight   today   cmafest   boston   nba   baby   rice   macndcheese   rib   cornonthecob')
('topic', 5, ':', u'today   drink   boston   12:30pm   sundayfunday   # boston   event   friyay   pizza   budlight')
('topic', 6, ':', u'cmafest   friyay   nashville   guinness   nyc   stellaartoris   setlife   hoegaarden   onset   profoto')
('topic', 7, ':', u'cmafest   nashville   kate   today   beer   like   cma   @countrymusic   @995wkdq   wkdq99.5')
('topic', 8, ':', u"budweiser   america   nats   drink   usmnt   usa   baseball   '   dc   gameday")
('topic', 9, '

For the above we used 10 topics so that we can show what is happening in a human readable format, but for the next steps we will want to use more topics

In [19]:
model = textacy.tm.TopicModel('lsa', n_topics=100)
model.fit(doc_term_matrix)
doc_topic_matrix = model.transform(doc_term_matrix)
print(doc_topic_matrix.shape)

(745, 100)


Now that we have our reduced matrix, we can do some clustering to see what hashtags are related

Because we don't want the user to have to choose the number of clusters, we have three clustering options from Sklearn that don't require a n_clusters paramter:
- DBSCAN generally performs better with more data
- MeanShift might perform well but is too computationally intensive
- Affinity Prograpagation is the most realistic option here

In [20]:
from sklearn.cluster import AffinityPropagation
ap = AffinityPropagation()
%time ap.fit(doc_topic_matrix)
clusters = ap.labels_.tolist()

CPU times: user 1.99 s, sys: 52.5 ms, total: 2.05 s
Wall time: 1.98 s


Now that we have the list of clusters, let's connect them with our hashtags to see what hashtags are being grouped together

In [21]:
df.head()

,hashtags,corpus
0,Canada,RT @FrankGirardot: If @Budweiser is now called...
1,Friday,"RT @KevinGarifo: It's #Friday, so must be time..."
2,justsayin,I need to get out of my own head before it suf...
3,dance,@budlight How can I win the @TheChainsmokers t...
4,hats,#MillerLite / #BudLight #hats for sale on #Eba...


In [22]:
df['clusters'] = clusters

In [23]:
df.head()

,hashtags,corpus,clusters
0,Canada,RT @FrankGirardot: If @Budweiser is now called...,59
1,Friday,"RT @KevinGarifo: It's #Friday, so must be time...",48
2,justsayin,I need to get out of my own head before it suf...,83
3,dance,@budlight How can I win the @TheChainsmokers t...,83
4,hats,#MillerLite / #BudLight #hats for sale on #Eba...,76


In [24]:
top_ten = df['clusters'].value_counts()[:10]

In [25]:
top_ten

83    118
84     53
34     20
78     17
10     16
56     15
28     14
58     14
71     13
18     12
Name: clusters, dtype: int64

In [26]:
top_ten.index

Int64Index([83, 84, 34, 78, 10, 56, 28, 58, 71, 18], dtype='int64')

In [27]:
for i in top_ten.index:
    print 'Cluster #: ' + str(i) + ' Hashtags: ' + str(top_ten.ix[i])
    print df['hashtags'][df.clusters == i].values[:10]

Cluster #: 83 Hashtags: 118
[u'justsayin' u'dance' u'jamaica' u'WestVsTheRest' u'WalmartSucks'
 u'WhatYeeKnow' u'securityissue' u'wetrepublic' u'sexyrexyshultz'
 u'Raiders']
Cluster #: 84 Hashtags: 53
[u'JH34' u'HappySunday' u'ribchi' u'HouseBoatofReps' u'youshouldbehere'
 u'lovesummer' u'toxic' u'drinkspecial' u'SportsStars' u'thestruts']
Cluster #: 34 Hashtags: 20
[u'PALS' u'YellowPissWater' u'weddingday' u'lathtr' u'hff16' u'boring'
 u'meetatthebar' u'marathontraining' u'soooohootttttt' u'QZOO']
Cluster #: 78 Hashtags: 17
[u'beberexha' u'yummy' u'PlayStation' u'wingnight' u'snoopdogg' u'LIT'
 u'oldschool' u'Spurs' u'budlightmusic' u'StanleyCup']
Cluster #: 10 Hashtags: 16
[u'FreebeeExperience' u'BudLightLime' u'budweiser' u'enjoyacoldone' u'DJ'
 u'COPA' u'copaam\xe9ricacentenario' u'minnows4life' u'PROGRESSIVEVIRUS'
 u'craftbeer']
Cluster #: 56 Hashtags: 15
[u'40oz' u'MaltLiquor' u'PuttPuttforPaws' u'lime' u'happytaco' u'lakelife'
 u'fitness' u'Merica' u'MonkeyBombs' u'merica']
Clus

Let's try this all again with DBSCAN

In [34]:
from sklearn.cluster import DBSCAN
dbs = DBSCAN()
%time dbs.fit(doc_topic_matrix)
clusters = dbs.labels_.tolist()
df['clusters'] = clusters
top_ten = df['clusters'].value_counts()[:10]
for i in top_ten.index:
    print 'Cluster #: ' + str(i) + ' Hashtags: ' + str(top_ten.ix[i])
    print df['hashtags'][df.clusters == i].values[:10]

CPU times: user 186 ms, sys: 6.73 ms, total: 193 ms
Wall time: 192 ms
Cluster #: 0 Hashtags: 276
[u'Canada' u'justsayin' u'dance' u'BuildABar' u'Pride' u'JH34' u'yummy'
 u'ShakersSaturday' u'pride' u'Cubs']
Cluster #: -1 Hashtags: 274
[u'Friday' u'hats' u'cooliecup' u'whatsmakeup' u'Pride2016' u'beberexha'
 u'swag' u'jamaica' u'rogerscentre' u'Thankyou']
Cluster #: 13 Hashtags: 13
[u'Ebay' u'eBay' u'Collectible' u'Discount' u'Sales' u'Basketball'
 u'Auction' u'Buzz' u'NBA' u'Detroit']
Cluster #: 3 Hashtags: 8
[u'PrinceTribute' u'PurpleRain' u'rocknroll' u'RealMenOfGenius' u'Legend'
 u'Cover' u'accoustic' u'singer']
Cluster #: 5 Hashtags: 8
[u'ThirstyThursday' u'jimmygreens' u'fireball' u'Fireball' u'special'
 u'thirstythursday' u'booze' u'sportsbar']
Cluster #: 1 Hashtags: 8
[u'SundayFunday' u'sundayfunday' u'atl' u'Pizza' u'pizza' u'ATL' u'Cobb'
 u'Draft']
Cluster #: 29 Hashtags: 7
[u'promo' u'Boston' u'boston' u'downtownboston' u'partybus' u'events'
 u'parade']
Cluster #: 26 Hashtags

In [36]:
from sklearn.metrics.pairwise import cosine_similarity
%time dist = 1 - cosine_similarity(doc_topic_matrix)
from sklearn.cluster import DBSCAN
dbs = DBSCAN(metric='precomputed')
%time dbs.fit(dist)
clusters = dbs.labels_.tolist()
df['clusters'] = clusters
top_ten = df['clusters'].value_counts()[:10]
for i in top_ten.index:
    print 'Cluster #: ' + str(i) + ' Hashtags: ' + str(top_ten.ix[i])
    print df['hashtags'][df.clusters == i].values[:10]

CPU times: user 11.3 ms, sys: 2.13 ms, total: 13.4 ms
Wall time: 12 ms
Cluster #: 0 Hashtags: 562
[u'Canada' u'Friday' u'justsayin' u'dance' u'hats' u'BuildABar'
 u'boersandbernsteinshow' u'whatsmakeup' u'Pride2016' u'Pride']
Cluster #: -1 Hashtags: 62
[u'cooliecup' u'yall' u'jackdaniels' u'jarritos' u'kmelsummerjam' u'twins'
 u'germanfest' u'finallyfuckinleft' u'ionaz'
 u'UMustStartDrinkingEarlyAzFuck']
Cluster #: 8 Hashtags: 13
[u'Ebay' u'eBay' u'Collectible' u'Discount' u'Sales' u'Basketball'
 u'Auction' u'Buzz' u'NBA' u'Detroit']
Cluster #: 16 Hashtags: 12
[u'Summer' u'ribfest2016' u'downforwhatever' u'summer' u'InThrees'
 u'LimeaRita' u'TheLodge' u'RibFest' u'surfyogabeer' u'yoga']
Cluster #: 3 Hashtags: 12
[u'jamaica' u'Pasadena' u'MexicoVsJamaica' u'Mexico' u'soccer' u'stencil'
 u'Rosebowl' u'mexico' u'AtlanticCity' u'doac']
Cluster #: 1 Hashtags: 12
[u'SundayFunday' u'WhatYeeKnow' u'sundayfunday' u'atl' u'Pizza' u'pizza'
 u'vezzo' u'ATL' u'LikeATank' u'Cobb']
Cluster #: 15 Hash

And once more with MeanShift

In [40]:
from sklearn.cluster import MeanShift
ms = MeanShift()
%time ms.fit(doc_topic_matrix)
clusters = ms.labels_.tolist()
df['clusters'] = clusters
top_ten = df['clusters'].value_counts()[:10]
for i in top_ten.index:
    print 'Cluster #: ' + str(i) + ' Hashtags: ' + str(top_ten.ix[i])
    print df['hashtags'][df.clusters == i].values[:10]

CPU times: user 2.54 s, sys: 61.4 ms, total: 2.6 s
Wall time: 2.78 s
Cluster #: 0 Hashtags: 722
[u'Canada' u'Friday' u'justsayin' u'dance' u'hats' u'SundayFunday'
 u'BuildABar' u'boersandbernsteinshow' u'cooliecup' u'whatsmakeup']
Cluster #: 3 Hashtags: 5
[u'rebelt5' u'teamcanon' u'canonrebel' u'beachmusic' u'northhills']
Cluster #: 2 Hashtags: 5
[u'GameDay' u'surfing' u'surf' u'sunshine' u'SunGlasses']
Cluster #: 1 Hashtags: 5
[u'mural' u'onmytravel' u'onmyrte66' u'Springfield' u'illinois']
Cluster #: 5 Hashtags: 4
[u'twins' u'Brazilian' u'Rio' u'Bitburger']
Cluster #: 4 Hashtags: 4
[u'fuckedup' u'cansorbottels' u'paperorplastic' u'themen']


Notes:
- 6/15 Used textacy and setup LSA, but it doesn't seem to return anything actionable, going to try clustering by cosine similarity

Connect tweets, tweeter counts, work on summarizer 

add user ids for direct marketing